##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.7.0


If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [28]:
generateLargeNetworkUntrained = True
if(generateLargeNetworkUntrained):
  #only train the last layer
  generateLargeNetworkRatio = 200
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128*generateLargeNetworkRatio, activation='relu'),
    tf.keras.layers.Lambda(lambda x: tf.keras.backend.stop_gradient(x)),
    tf.keras.layers.Dense(10)
  ])
else:
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
  ])

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [29]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.14759071,  0.11032473,  0.01213703, -0.0391947 , -0.20604946,
        -0.09968031, -0.00402072, -0.02173454, -0.02368924, -0.00212337]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [30]:
tf.nn.softmax(predictions).numpy()

array([[0.11687003, 0.11259491, 0.10206489, 0.09695791, 0.08205767,
        0.09126719, 0.10042901, 0.09866568, 0.09847301, 0.10061973]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [31]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [32]:
loss_fn(y_train[:1], predictions).numpy()

2.3939638

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [33]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [34]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 81s 43ms/step - loss: 0.2567 - accuracy: 0.9277
Epoch 2/5
1875/1875 [==============================] - 80s 43ms/step - loss: 0.1304 - accuracy: 0.9617
Epoch 3/5
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0930 - accuracy: 0.9733
Epoch 4/5
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0719 - accuracy: 0.9793
Epoch 5/5
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0569 - accuracy: 0.9838


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [35]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 12s - loss: 0.0771 - accuracy: 0.9758 - 12s/epoch - 37ms/step


[0.07711564749479294, 0.9757999777793884]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [36]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [37]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[7.7854487e-08, 1.8036658e-09, 2.2356211e-05, 5.2717159e-04,
        1.8274392e-08, 8.1863789e-07, 2.6288660e-10, 9.9940991e-01,
        1.1263585e-06, 3.8532555e-05],
       [6.5186382e-06, 1.2741956e-05, 9.9970764e-01, 1.2508495e-04,
        6.9836831e-11, 3.7782200e-05, 2.1325679e-05, 8.4702685e-14,
        8.8806002e-05, 6.8828637e-11],
       [1.4661248e-06, 9.9499834e-01, 4.9324310e-04, 8.2308456e-05,
        5.1083119e-04, 7.7333883e-05, 1.0240673e-05, 2.7641344e-03,
        9.8173588e-04, 8.0297723e-05],
       [9.9990189e-01, 2.9844360e-09, 5.0838487e-05, 2.4028018e-06,
        4.0202013e-07, 1.0214205e-06, 2.2963857e-05, 8.8982606e-06,
        4.2368409e-08, 1.1640392e-05],
       [1.2222447e-05, 3.9165485e-08, 6.9855560e-05, 7.0760058e-07,
        9.9342936e-01, 1.8129228e-06, 1.0468366e-04, 1.6348215e-04,
        6.6751713e-06, 6.2111453e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
